In [3]:
import requests
from bs4 import BeautifulSoup
import re
import os
import urllib3

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
downloads = 0

def download_pdf(pdf_url):
    global downloads
    response = requests.get(pdf_url, verify=False)
    pattern = re.compile(r'/index\.php/([^/]+)')
    pdf_dir = re.search(pattern, pdf_url).group(1)
    pdf_name = pdf_url.split('/')[-1] + '.pdf'
    pdf_path = os.path.join(pdf_dir, pdf_name)
    os.makedirs(pdf_dir, exist_ok=True)
    with open(pdf_path, 'wb') as f:
        f.write(response.content)
    
    downloads += 1


def download_pdfs(archive_url):
    pg_no = 1
    page_url = f'{archive_url}/{pg_no}'
    while(True):
        print(f'scrapping page: {page_url}')
        try :
            response = requests.get(page_url, verify=False)
            soup = BeautifulSoup(response.text, 'html.parser')
        except:
            print("error while scrapping {page_url}, skipping")
            continue
        
        issue_links = [a['href'] for a in soup.find_all('a', href=True) if 'issue/view' in a['href']]
        print('got {len(issue_links) }')
        if(len(issue_links) == 0 ):
            break
        pg_no+=1
        page_url = f'{archive_url}/{pg_no}'
        
        for issue_link in issue_links:
            issue_response = requests.get(issue_link, verify=False)
            issue_soup = BeautifulSoup(issue_response.text, 'html.parser')
            
            pdf_links = [a['href'] for a in issue_soup.find_all('a', href=True) if 'article/view' in a['href']]

            for pdf_link in pdf_links:
                pdf_response = requests.get(pdf_link, verify=False)
                pdf_soup = BeautifulSoup(pdf_response.text, 'html.parser')
                
                download_links = [a['href'] for a in pdf_soup.find_all('a', href=True) if 'article/download' in a['href']]
                for download_link in download_links:
                    download_pdf(download_link)



In [ ]:

# we will just skip these, may be scrape them with selenium in future
login_req_list = ['https://epubs.icar.org.in/index.php/IJHF/issue/archive', 'https://epubs.icar.org.in/index.php/Agropedology/issue/archive',
                  'https://epubs.icar.org.in/index.php/IJVASR/issue/archive', 'https://epubs.icar.org.in/index.php/JIFSI/issue/archive',
                  'https://epubs.icar.org.in/index.php/TJRP/issue/archive', 'https://epubs.icar.org.in/index.php/IJAN/issue/archive', 
                  'https://epubs.icar.org.in/index.php/OIJR/issue/archive', 'https://epubs.icar.org.in/index.php/JSWC/issue/archive',
                  'https://epubs.icar.org.in/index.php/IJPP/issue/archive', 'https://epubs.icar.org.in/index.php/IPPJ/issue/archive',
                  'https://epubs.icar.org.in/index.php/jasa/issue/archive', 'https://epubs.icar.org.in/index.php/APPS/issue/archive',
                  'https://epubs.icar.org.in/index.php/VIB/issue/archive', 'https://epubs.icar.org.in/index.php/AERR/issue/archive',
                  'https://epubs.icar.org.in/index.php/JCMSD/issue/archive', 'https://epubs.icar.org.in/index.php/JAEM/issue/archive',
                  'https://epubs.icar.org.in/index.php/TR/issue/archive']

In [4]:

base_url = 'https://epubs.icar.org.in'


base_pg_soup = BeautifulSoup(requests.get(base_url, verify=False).text, 'html.parser')
journal_links = [a['href'] for a in base_pg_soup.find_all('a', href=True) if '/issue/current' in a['href']]
archive_links = [link.replace('/current', '/archive') for link in journal_links]
archive_links
# for archive_link in archive_links:
#     download_pdfs(archive_link)

# https://epubs.icar.org.in/index.php/IJPP/issue/archive

['https://epubs.icar.org.in/index.php/IJAgS/issue/archive',
 'https://epubs.icar.org.in/index.php/IJAnS/issue/archive',
 'https://epubs.icar.org.in/index.php/IJF/issue/archive',
 'https://epubs.icar.org.in/index.php/FT/issue/archive',
 'https://epubs.icar.org.in/index.php/JoSSWQ/issue/archive',
 'https://epubs.icar.org.in/index.php/CoRJ/issue/archive',
 'https://epubs.icar.org.in/index.php/PotatoJ/issue/archive',
 'https://epubs.icar.org.in/index.php/IJEE/issue/archive',
 'https://epubs.icar.org.in/index.php/IndFarm/issue/archive',
 'https://epubs.icar.org.in/index.php/IndHort/issue/archive',
 'https://epubs.icar.org.in/index.php/JMAP/issue/archive',
 'https://epubs.icar.org.in/index.php/JOR/issue/archive',
 'https://epubs.icar.org.in/index.php/JAE/issue/archive',
 'https://epubs.icar.org.in/index.php/TJRA/issue/archive',
 'https://epubs.icar.org.in/index.php/JISSS/issue/archive',
 'https://epubs.icar.org.in/index.php/IJVA/issue/archive',
 'https://epubs.icar.org.in/index.php/IJDS/issu

In [ ]:
print(f"downloaded {downloads} pdfs")

: 